In [164]:
VERSION = 33

# О проекте

Мы работаем в компании, которая занимается продажей автомобилей с пробегом. Основная задача компании и ее менеджеров - максимально быстро находить выгодные предложения (проще говоря, купить ниже рынка, а продать дороже рынка).

Нашей команде ([Денис Волков](https://sfdatasciencecourse.slack.com/team/US19J2A64), [Максим Камашев](https://sfdatasciencecourse.slack.com/team/URVGMMG0L) и [Юрий Бикузин](https://sfdatasciencecourse.slack.com/team/U016P0Y3CP7)) поставлена задача создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам.
Если наша модель работает хорошо, то мы сможем быстро выявлять выгодные предложения (когда желаемая цена продавца ниже предсказанной рыночной цены). Это значительно ускорит работу менеджеров и повысит прибыль компании.

Для проверки нашей модели заказчик придумал нам испытание. Он подготовил [тестовый датасет](https://drive.google.com/u/0/uc?id=18dDPo6GF5VSU2MaIvOk6PFjUnfBi3A42&export=downloa) с таким набором параметров, который будет использоваться для оценки по нашей модели

Мы решили внимательно исследовать тестовый датасет, чтобы при подготовить данные, совместимые с ним
Для сбора данных мы заранее наметили сайт [Авито](https://www.avito.ru/moskva/transport?cd=1), для которого написали [сканер](https://github.com/yurybikuzin/avito_scanner)

## Датасет

Загрузим трейновый датасет с данными, полученными с Авито:

In [166]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

is_debug = False
if not is_debug:
    # train_dataset_url="https://drive.google.com/u/0/uc?id=1HFV1106xXhrnNt5wG1nXl0X-b5Jql2Md&export=download"
    train_dataset_url="https://drive.google.com/u/0/uc?id=1MaX59-keo_h4TEGKwW-HO7Ehh_O2wVwu&export=download"
    train_orig = pd.read_csv(train_dataset_url, low_memory = False)
train = train_orig.copy()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60028 entries, 0 to 60027
Data columns (total 77 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   bodyType                              59979 non-null  object 
 1   brand                                 59979 non-null  object 
 2   color                                 60028 non-null  object 
 3   fuelType                              59979 non-null  object 
 4   name                                  59979 non-null  object 
 5   title                                 0 non-null      float64
 6   numberOfDoors                         59977 non-null  float64
 7   productionDate                        60024 non-null  float64
 8   vehicleTransmission                   59979 non-null  object 
 9   engineDisplacement                    59682 non-null  object 
 10  enginePower                           60012 non-null  object 
 11  description    

Загрузим тестовый датасет (мы его предварительно разместили на Google Drive, чтобы к нему можно было обращаться по ссылке):

In [168]:
if not is_debug:
    test_dataset_url="https://drive.google.com/u/0/uc?id=18dDPo6GF5VSU2MaIvOk6PFjUnfBi3A42&export=download"
    test_orig = pd.read_csv(test_dataset_url)
test = test_orig.copy()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837 entries, 0 to 3836
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              3837 non-null   object 
 1   brand                 3837 non-null   object 
 2   color                 3837 non-null   object 
 3   fuelType              3837 non-null   object 
 4   modelDate             3837 non-null   float64
 5   name                  3837 non-null   object 
 6   numberOfDoors         3837 non-null   float64
 7   productionDate        3837 non-null   float64
 8   vehicleConfiguration  3837 non-null   object 
 9   vehicleTransmission   3837 non-null   object 
 10  engineDisplacement    3837 non-null   object 
 11  enginePower           3837 non-null   object 
 12  description           3837 non-null   object 
 13  mileage               3837 non-null   float64
 14  Комплектация          3837 non-null   object 
 15  Привод               

В нем только одно частично заполненное поле: `Владение`, все остальные поля полностью заполнены

Целью нашего рассмотрения тестового и трейнового датасетов будет отбор таких полей из обоих, чтобы можно было скомпоновать единый датасет на основе теста и трейна, с общим набороом колонок

При формировании колонок единого датасета тон будет задавать именно тестовый датасет, потому что он содержит поля, по которым закачик будет проверять валидность нашей модели и далее использовать ее на практике

После формирования единого датасета мы уже из него выделим новый трейновый, обучим на нем модель, а потом прогоним на ней новой тестовый датасет, выделенный из единого, чтобы получить submission для kaggle, где заказчик будет проверять валидность построенной нами модели

Поэтому далее мы просматриваем поля именно тестового датасета, пытаясь подобрать подходящие данные из трейнового

В переменной `common_cols` будем накапливать список общих колонок для единого датасета:

In [ ]:
common_cols = set()

Определим вспомогательную функцию:

In [ ]:
def describe(df, field_name):
    print(f"Колонка \"{field_name}\":")
    print("------")
    print("na:", df[field_name].isna().sum())
    print("уникальных значений:", len(df[field_name].unique()))
    print("------")
    print(df[field_name].value_counts())  

### bodyType

In [ ]:
df = test
field_name = 'bodyType'
describe(df, field_name)

Обратим внимание на то, что есть значения, например "xэтчбек 5 дв.", который совпадают с другим значением "хэтчбек", но имеют дополнительно указание количества дверей

Поскольку у нас есть отдельное полей `numberOfDoors`, содержащее количество дверей, то давайте проверим, есть ли расхождение в количестве дверей в этом поле со значением, указанным в поле bodyType:

In [ ]:
def extract(x):
    ss = str(x).split()
    if len(ss) > 2:
        return float(ss[1])
    else:
        return None
df['дв'] = df[field_name].apply(lambda x: extract(x))
print(df[ (df['дв'].notna()) & (df['дв'] != df.numberOfDoors) ]['дв'].count())
df.drop(['дв'], inplace = True, axis = 1)

Видим, что расхождений нет, а значит составные значений поля `bodyType`, такие как "хэтчбек 5 дв.", содержат информацию, дублирующую в поле `numberOfDoors`, а значит, мы можем очистить составные значения поля `bodyType` от этой дублирующей информации и сократить количество разных значений поля:

In [ ]:
df[field_name] = df[field_name].apply(lambda x: str(x).split()[0])
describe(df, field_name)

В трейне есть колонка с тем жем названием:

In [ ]:
describe(train, field_name)

Однако набор значений немного отличается:

In [ ]:
a = set(train[train[field_name].notna()][field_name].unique().tolist())
b = set(test[field_name].unique().tolist())
print("Общие значения:", a.intersection(b))
print("Значения, которых нет в тесте:", a - b)
print("Значения, которых нет в трейне:", b - a)

"лифтбек" - это [разновидность](https://avtovikup136.ru/topics/%D1%82%D0%B8%D0%BF-%D0%BA%D1%83%D0%B7%D0%BE%D0%B2%D0%B0-%D0%BB%D0%B8%D1%84%D1%82%D0%B1%D0%B5%D0%BA-%D1%87%D1%82%D0%BE-%D1%8D%D1%82%D0%BE/) "хэтчбека"

"компактвэн" и "минивэн" - это [достаточно близкие](https://autozam.ru/klassi-avtomobiley/odin-na-semerich-mini-i-kompaktveni.html) понятия

а "родстер" - "Термин часто используется просто как коммерческое название двухдверного двухместного кабриолета" ([источник](https://ru.wikipedia.org/wiki/%D0%A0%D0%BE%D0%B4%D1%81%D1%82%D0%B5%D1%80))

Произведем соответствующие замены:

In [ ]:
def unify(x):
    if x == "лифтбек":
        return "хэтчбек"
    elif x == "компактвэн":
        return "минивэн"
    elif x == "родстер":
        return "кабриолет"
    else:
        return x
df[field_name] = df[field_name].apply(unify)
describe(df, field_name)
a = set(train[train[field_name].notna()][field_name].unique().tolist())
b = set(test[field_name].unique().tolist())
print("Общие значения:", a.intersection(b))
print("Значения, которых нет в тесте:", a - b)
print("Значения, которых нет в трейне:", b - a)

Таким образом, мы привели колонки `bodyType` теста и трейна к общему знаменателю (трейна)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### brand

In [ ]:
field_name = 'brand'
describe(df, field_name)

Видим, что тестовая выборка монобрендовая. Однако мы будем готовить полибрендовые данные для обучения, так как мы помним, что наша задача - создать модель для быстрого выкупа автомобилей, а тестовая выборка предназначена лишь для проверки нашей модели, сама же модель будет использоватся для разных брендов

В трейне есть поле с тем же названием:

In [ ]:
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### color

In [ ]:
field_name = 'color'
describe(df, field_name)

В трейне есть поле с тем же названием:

In [ ]:
describe(train, field_name)

Для приведения к общему знаменателю достаточно перевести значения в трейне в нижний регистр:

In [ ]:
train[field_name] = train[field_name].apply(lambda x: None if pd.isna(x) else x.lower())
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### fuelType

In [ ]:
field_name = 'fuelType'
describe(df, field_name)

В трейне есть поле с тем же названием:

In [ ]:
describe(train, field_name)

Для приведения к общему знаменателю достаточно перевести значения в трейне в нижний регистр:

In [ ]:
train[field_name] = train[field_name].apply(lambda x: None if pd.isna(x) else x.lower())
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### modelDate

In [ ]:
field_name = 'modelDate'
describe(df, field_name)

Поля с тем же именем в трейне нет, но есть поле `autocatalogWorldPremier`:

In [ ]:
describe(train, 'autocatalogWorldPremier')

Из которой можно извлечь нужную информацию:

In [ ]:
def extract(x):
    found_year = re.search('(\d\d\d\d)', x)
    if found_year:
        return int(found_year.group(1))
    else:
        return None   
train[field_name] = train['autocatalogWorldPremier'].apply(lambda x: None if pd.isna(x) else extract(x))
describe(train, field_name)

Но, к сожалению, в этом поле слишком много пропусков, поэтому мы вынуждены его проигнорировать

### name

In [ ]:
field_name = 'name'
describe(df, field_name)

Видим, что поле `name` содержит составные значения

Попробуем разобрать эти значения на запчасти

Сначала выделим признак полного привода в отдельное поле:

In [ ]:
df['4WD'] = df[field_name].apply(lambda x: 1 if "4WD" in x else 0)

А отметку этого признака уберем из поля `name`:

In [ ]:
df[field_name] = df[field_name].apply(lambda x: x[:-4] if x.endswith(' 4WD') else x)
describe(df, field_name)

Затем обратим внимание на то, что у нас есть отдельное поле `enginePower`, тоже содержащее количество лошадиных сил, указанное в скобках в значении рассматриваемого нами поля `name`

In [ ]:
df.enginePower.unique()

Давайте убедимся, что количество лошадиных сил в поле `name` просто дублирует значение поля `enginePower`, проверив, есть ли расхождения в значениях:

In [ ]:
A = 'л.с. из enginePower'
B = 'л.с. из name'
df[A] = df.enginePower.apply(lambda x: int(x.split()[0]))
def extract(x):
    found_engine_power = re.search('\((\d+)\s*л\.\с\.\)', x)
    if found_engine_power:
        return int(found_engine_power.group(1))
    else:
        return None
df[B] = df[field_name].apply(lambda x: extract(x))
print("Расхождений в лошадиных силах:", df[ (df[B].notna()) & (df[B] != df[A]) ][B].count())
df.drop([A, B], inplace = True, axis = 1)

Как видим, расхождений нет, поэтому смело можем убрать количество лошадиных сил из значений поля `name`, так как для этого есть отдельное поле `enginePower`, которое содержит совпадающее значение:

In [ ]:
def drop_suffix(x):
    x = re.sub('\s*\((\d+)\s*л\.\с\.\)$', '', x)
    return x
df[field_name] = df[field_name].apply(lambda x: drop_suffix(x))
describe(df, field_name)

У нас осталось одно значение в поле `name`, в котором мощность указана не лошадиных силах, а в киловаттах: "Electro AT (126 кВт)"

Давайте посмотрим, что указано в поле enginePower для этой записи:

In [ ]:
df[(df[field_name].str.contains('кВт'))].enginePower

Указана мощность в 170 лошадиных сил, но это, согласно [калькулятору перевода лошадиных сил в киловатты](https://www.google.com/search?q=%D0%BA%D0%B8%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%82%D1%8B+%D0%B2+%D0%BB%D0%BE%D1%88%D0%B0%D0%B4%D0%B8%D0%BD%D1%8B%D0%B5+%D1%81%D0%B8%D0%BB%D1%8B&oq=%D0%BA%D0%B8%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%82%D1%8B), в точности совпадает со 126 кВт мощности, указанному в значении поля `name`

Поэтому и в этом случае можно очистить значение поля `name` от дублирующей информации:

In [ ]:
def drop_suffix(x):
    x = re.sub('\s*\((\d+)\s*кВт\)$', '', x)
    return x
df[field_name] = df[field_name].apply(lambda x: drop_suffix(x))
describe(df, field_name)

Теперь обратим внимание на суффикс "AT"/"МТ"

Если соотнести его присутствие в значении поля `name` со значеним поля `vehicleTransmission`:

In [ ]:
print("Трансмиссии: ", df.vehicleTransmission.unique())
df[df[field_name].str.contains("[AM]T", regex=True)][[field_name, 'vehicleTransmission']]

То можно предположить, что этот суффикс дублирует информацию из поля `vehicleTransmission`
Давайте проверим это предположение, проверив отсутствие расхожений значения трансмисии, извлеченной из поля `name`, со значением, указанным в поле `vehicleTransmission`:

In [ ]:
A = 'трансмиссия из поля name'
df[A] = df[field_name].apply(lambda x: 'автоматическая' if "AT" in x else 'механическая' if "MT" in x else None)
print("Расхождений в трансмиссии:", df[ (df[A].notna()) & (df[A] != df.vehicleTransmission) ][A].count())

Упс! Обнаружены расхождения

Значит, наше предположение о полном соответствии трансмиссии, указанной в поле `name`, со значением в поле 'vehicleTransmission' неверно

Или неверно лишь в том виде, в котором мы его сделали

Давайте проверим строки, в которых это расхождение обнаружено:

In [ ]:
print(df[ (df[A].notna()) & (df[A] != df.vehicleTransmission) ][['name', 'vehicleTransmission']])

Можно увидеть, что есть третий суффикс "AMT", который мы не выделили сразу, но он, похоже, соответствует значению трансмиссии "роботизированная". Выходит, наше изначальное предположение было лишь неполным, а не неверным

Давайте проверим, что есть соответствие "AT" => "автоматическая", "МТ" => "механическая", "АМТ" => "роботизированная":

In [ ]:
A = 'трансмиссия из поля name'
def extract(x):
    if "AMT" in x:
        return 'роботизированная'  
    elif "AT" in x:
        return 'автоматическая'
    elif "MT" in x:
        return 'механическая'
    else:
        return None    
df[A] = df[field_name].apply(lambda x: extract(x))
print("Расхождений в трансмиссии:", df[ (df[A].notna()) & (df[A] != df.vehicleTransmission) ][A].count())
df.drop([A], inplace = True, axis = 1)

Расхождений нет

Значит наше предполжение оправдано, и можно очистить значение поля `name` от лишнего суффикса:

In [ ]:
def drop_suffix(x):
    x = re.sub('\s[AM]+T$', '', x)
    return x
df[field_name] = df[field_name].apply(lambda x: drop_suffix(x))
describe(df, field_name)

Обратим внимание на суффикс, например "3.0d", похожий на вещественное число, иногда сопровождаемый буквенным кодом

Давайте посмотрим, какими еще буквами сопровождается вещественное число в значении поля `name`:

In [ ]:
def extract(x):
    found_suffix = re.search('\d+\.\d(\w+)$', x)
    if found_suffix:
        return found_suffix.group(1)
    else:
        return None
    
list(set(filter(lambda x: x is not None, map(extract, df[field_name].unique().tolist()))))

Если соотнести буквенный код со значениями в поле `fuelType`):

In [ ]:
A = 'code'
df[A] = df[field_name].apply(lambda x: extract(x))
print(df[ (df[A].notna()) & (df[A] == 'd') ][[A, 'fuelType']])
print(df[ (df[A].notna()) & (df[A] == 'hyb') ][[A, 'fuelType']])

То можно увидеть, что код, если указан, всегда дублирует значение поля `fuelType`

Проверим это предположение:

In [ ]:
B = 'fuelType по коду'
df[B] = df[A].apply(lambda x: 'дизель' if x == 'd' else 'гибрид' if x == 'hyb' else None)
print("Расхождений в типе топлива:", df[ (df[B].notna()) & (df[B] != df.fuelType) ][B].count())
df.drop([A, B], inplace = True, axis = 1)

Если соотнести вещественное число со значениями полей `engineDisplacement` (объем двигателя):

In [ ]:
df[[field_name, 'engineDisplacement']]

То можно заметить соответствие этого вещественного числа значению поля `engineDisplacement`

Проверим этo предположения:

In [ ]:
A = 'объем двигателя из name'
B = 'объем двигателя из engineDisplacement'

def extract_engine_displacement(x, regex):
    found_engine_displacement = re.search(regex, x)
    if found_engine_displacement:
        return float(found_engine_displacement.group(1))
    else:
        return None
df[A] = df[field_name].apply(lambda x: extract_engine_displacement(x, '\s*(\d+\.\d)(\D.*)$'))
df[B] = df.engineDisplacement.apply(lambda x: extract_engine_displacement(x, '^(\d+\.\d)'))
print("Расхождений в объеме двигателя:", df[ (df[A].notna()) & (df[A] != df[B]) ][A].count())
df[ (df[A].notna()) & (df[A] != df[B]) ][[A, B]]
df.drop([A, B], inplace = True, axis = 1)

Обнаружено одно расхождение, но им в данном случае можно пренебречь (оно в пределах погрешности), и очистить поле `name` от суффикса с вещественным числом с буквенным кодом

Следовательно, можно очистить значение поля `name` и от суффикса в виде вещественного числа с буквенным кодом:

In [ ]:
def drop_suffix(x):
    x = re.sub('\s*\d+\.\d\w*$', '', x)
    return x
df[field_name] = df[field_name].apply(lambda x: drop_suffix(x))
describe(df, field_name)

Проверим оставшиеся уникальные значения

In [ ]:
df[field_name].unique()

Обратим на присутствие кодов "xDrive" и "sDrive". "xDrive" - это код [системы интеллектуального полного привода](https://techautoport.ru/transmissiya/sistemy-polnogo-privoda/xdrive.html), а "sDrive" - это код системы заднего привода

Вынесем коды привода в отдельные поля:

In [ ]:
df['xDrive'] = df[field_name].apply(lambda x: 1 if "xDrive" in x else 0)
df['sDrive'] = df[field_name].apply(lambda x: 1 if "sDrive" in x else 0)

И очистим значение поля `name` от этого кода

In [ ]:
def drop_suffix(x):
    x = re.sub('[xs]Drive', '', x)
    return x
df[field_name] = df[field_name].apply(lambda x: drop_suffix(x))
describe(df, field_name)
df[field_name].unique()

В результате в поле `name` у нас остались модели BMW

In [ ]:
df['model'] = df[field_name]

В трейне поле `name` содержит другую информацию:

In [ ]:
describe(train, field_name)

Информацию похожую на искомую содержит поле `autocatalogTitle`:

In [ ]:
train[(train.brand == "BMW")]['autocatalogTitle'].unique()

Произведем обработку трейне, аналогичную той, что мы произвели для рассматриваемого поля в тесте:

In [ ]:
df_temp = df
field_name_temp = field_name
try:
    df = train
    field_name = 'autocatalogTitle'
#     df[field_name] = train_orig[field_name]
    def drop_suffix(x):
        x = re.sub('\s*\((\d+)\s*л\.\с\.?\s*\)', '', x)
        x = re.sub('\s[AM]+T', '', x)
        return x
    df[field_name] = df[field_name].apply(lambda x: None if pd.isna(x) else drop_suffix(x))
    df['4WD'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and 'AWD' in x else 0)
    df['FWD'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and 'FWD' in x else 0)
    df['RWD'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and 'RWD' in x else 0)
    df['4WD'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and re.match(r'quattro|4(?:WD|MATIC|x?Motion)', x, flags=re.IGNORECASE) else 0)
    df['xDrive'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and "xDrive" in x else 0)
    df['sDrive'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and "sDrive" in x else 0)
    df['eDrive'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and "eDrive" in x else 0)
    df['tronic'] = df[field_name].apply(lambda x: 1 if pd.notna(x) and ("tronic" in x or "Tronic" in x) else 0)
    def drop_suffix(x):
        x = re.sub('\s*[ARF]WD', '', x)
        x = re.sub('\s*4(x?motion|matic|wd)', '', x, flags=re.IGNORECASE)
        x = re.sub('\s*quattro', '', x)
        x = re.sub('\s*[xes]Drive', '', x)
        x = re.sub('\s*(?:Step|Tip|Multi)tronic', '', x)
        x = re.sub('\s*\d+G-Tronic', '', x, flags=re.IGNORECASE)
        return x
    df[field_name] = df[field_name].apply(lambda x: None if pd.isna(x) else drop_suffix(x))
    def drop_suffix(x):
        x = re.sub('\s*\d+\.\d\w*$', '', x)
        return x
    df[field_name] = df[field_name].apply(lambda x: None if pd.isna(x) else drop_suffix(x))
    print("Получившиеся модели BMW:", df[(df.brand == "BMW")][field_name].unique().tolist())
    print("------------------------")
    print("Получившиеся модели остальных брендов:", df[(df.brand != "BMW")][field_name].unique().tolist())
finally:
    df = df_temp
    field_name = field_name_temp

Также поместим полученный результат в поле `model`:

In [ ]:
train['model'] = train['autocatalogTitle']

Добавим в список колонок единого датасета следующие колонки: `model`, `xDrive`, `sDrive`, `4WD`

In [ ]:
for field_name in ['model', 'xDrive', 'sDrive', '4WD']:
    common_cols.add(field_name)
common_cols

### numberOfDoors

In [ ]:
field_name = 'numberOfDoors'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### productionDate

In [ ]:
field_name = 'productionDate'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### vehicleConfiguration

In [ ]:
field_name = 'vehicleConfiguration'
describe(df, field_name)
df[field_name].unique()

Здесь мы, как и в поле `name`, наблюдаем составное значение

Очень похоже, что последнее вещественное числов значение - это объем двигателя, который есть в отдельном поле `engineDisplacement`, "AUTOMATIC"/"MECHANICAL" - это тип трансмиссии, который есть в отдельном поле `vehicleTransmission`, а первое "слово" в этом составном значении - это тип кузова (`bodyType`) дополненный значением `numberOfDoors`

Проверим наше предположение:

In [ ]:
A = 'объем двигателя из vehicleConfiguration'
B = 'объем двигателя из engineDisplacement'
C = 'трансмиссия из vehicleConfiguration'
D = 'количество дверей из vehicleConfiguration'
E = 'тип кузова из vehicleConfiguration'
def extract_engine_displacement(x, regex):
    found_engine_displacement = re.search(regex, x)
    if found_engine_displacement:
        return float(found_engine_displacement.group(1))
    else:
        return None
df[A] = df[field_name].apply(lambda x: extract_engine_displacement(x, '\s*(\d+\.\d)(\D.*)$'))
df[B] = df.engineDisplacement.apply(lambda x: extract_engine_displacement(x, '^(\d+\.\d)'))
print("Расхождений в объеме двигателя:", df[ (df[A].notna()) & (df[A] != df[B]) ][A].count())
print("Варианты трансмиссии из vehicleConfiguration:", set(list(map(lambda x: x.split()[1], df[field_name].unique()))))
def extract_vehicle_transmission(x):
    ss = x.split()
    if len(ss) < 2:
        return None
    else:
        s = ss[1]
        if s == 'ROBOT':
            return 'роботизированная'
        elif s == 'MECHANICAL':
            return 'механическая'
        elif s == 'AUTOMATIC':
            return 'автоматическая'
        else:
            return 'Unknown'
df[C] = df[field_name].apply(lambda x: extract_vehicle_transmission(x))
print("Расхождений в трансмиссии:", df[ (df[C].notna()) & (df[C] != df.vehicleTransmission) ][C].count())
def extract_number_of_doors(x):
    ss = x.split()
    if len(ss) < 1:
        return None
    else:
        s = ss[0]
        found_number_of_doors = re.search('_(\d+)_DOORS', s)
        if found_number_of_doors:
            return int(found_number_of_doors.group(1))
        else:
            return None
df[D] = df[field_name].apply(lambda x: extract_number_of_doors(x))
print("Расхождений в количестве дверей:", df[ (df[D].notna()) & (df[D] != df.numberOfDoors) ][D].count())
df[D].unique()
def extract_body_type(x):
    ss = x.split()
    if len(ss) < 1:
        return None
    else:
        s = ss[0]
        ss = s.split('_')
        if len(ss) < 1:
            return None
        else:
            return ss[0]
print("Варианты типа кузова из vehicleConfiguration:", 
      set(list(map(lambda x: extract_body_type(x), df[field_name].unique())))
)
def extract_body_type_rus(x):
    s = extract_body_type(x)
    if s is None:
        return None
    elif s == 'ROADSTER':
        return 'родстер'
    elif s == 'HATCHBACK':
        return 'хэтчбек'
    elif s == 'LIFTBACK':
        return 'лифтбек'
    elif s == 'COMPACTVAN':
        return 'компактвэн'
    elif s == 'CABRIO':
        return 'кабриолет'
    elif s == 'SEDAN':
        return 'седан'
    elif s == 'COUPE':
        return 'купе'
    elif s == 'WAGON':
        return 'универсал'
    elif s == 'ALLROAD':
        return 'внедорожник'
    else:
        return 'Unknown'
df[E] = df[field_name].apply(lambda x: extract_body_type_rus(x))
print("Расхождений в типе кузова:", df[ (df[E].notna()) & (df[E] != df.bodyType) ][E].count())
df.drop([A, B, C, D, E], inplace = True, axis = 1)

Наши предположения полностью подтвердились

Это значит, что поле `vehicleConfiguration` не информативно, и его можно отбросить за ненадобностью:

In [ ]:
df.drop([field_name], inplace = True, axis = 1)

### vehicleTransmission

In [ ]:
field_name = 'vehicleTransmission'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Доработаем трейн:

In [ ]:
train[field_name] = train[field_name].apply(lambda x: 'роботизированная' if pd.notna(x) and x == 'робот' else x)
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### engineDisplacement

In [ ]:
field_name = 'engineDisplacement'
describe(df, field_name)

Уберем неинформативный суффикс "LTR":

In [ ]:
df[field_name] = df[field_name].apply(lambda x: x.split()[0])
describe(df, field_name)

In [ ]:
df[df[field_name] == 'undefined']

Мы не можем заменить значение "undefined" ни на какой объем двигателя, потому что в данном случае речь идет об электрокаре

В трейне есть аналогичное поле:

In [ ]:
describe(train, field_name)

Здесь есть только одно специфическое значение "6.0+":

In [ ]:
train[field_name] = train[field_name].apply(lambda x: "6.0" if pd.notna(x) and x == "6.0+" else x)
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### enginePower

In [ ]:
field_name = 'enginePower'
describe(df, field_name)

Уберем неинформативный суффикс "LTR":

In [ ]:
df[field_name] = df[field_name].apply(lambda x: x.split()[0])
describe(df, field_name)

В трейне есть аналогичное поле:

In [ ]:
describe(train, field_name)

Здесь нужно убрать ненужный суффикс "л.с.":

In [ ]:
def drop_suffix(x):
    x = re.sub(r'\s*л.с\.?\s*$', '', x)
    return x
train[field_name] = train[field_name].apply(lambda x: None if pd.isna(x) else drop_suffix(x))
train[field_name].unique()

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### description

In [ ]:
field_name = 'description'
describe(df, field_name)

Непонятно, что полезного можно почерпнуть из этого поля

Убираем из дальнейшего рассмотрения:

In [ ]:
df.drop([field_name], inplace = True, axis = 1)

### mileage

In [ ]:
field_name = 'mileage'
describe(df, field_name)

In [ ]:
describe(train, 'mileage')

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)
common_cols

### Комплектация

In [ ]:
field_name = 'Комплектация'
describe(df, field_name)

Поле содержит json, значения из которого мы извлечем:

In [ ]:
import json
features = set()
for s in df[field_name].unique().tolist():
    if len(s) > 4:
        decoded = json.loads(s[2:-2])
        for segment in decoded:
            name = segment['name']
            for feature in segment['values']:
                features.add(feature + '::' + name)
print("Количество разных атрибутов комлектации:", len(features))
print("Список атрибутов комлектации:")
print("------------------------------")
for feature in sorted(list(features)):
    print(feature)

Все эти атрибуты можно превратить в отдельные boolean (0|1) поля:

In [ ]:
def extract_feature(feature, s):
    (feature_name, segment_name) = feature.split('::')
    if len(s) > 4:
        decoded = json.loads(s[2:-2])
        for segment in decoded:
            if segment['name'] == segment_name:
                for feature in segment['values']:
                    if feature == feature_name:
                        return 1
    return 0
for feature in sorted(list(features)):
    df[feature] = df[field_name].apply(lambda x: 0 if pd.isna(x) else extract_feature(feature, x))
    

In [ ]:
df.info(verbose=True)

Проверим несколько новых полей:

In [ ]:
for feature in ['Яндекс.Авто::Мультимедиа', 'Фаркоп::Прочее', 'Сиденья с массажем::Салон']:
    print(df[feature].value_counts())
    print("")

В трейне про комлектацию есть следующие поля: `Электростеклоподъемники`, `Усилитель руля`, `Аудиосистема`, `Фары`, `Климат-контроль`, `Салон`, `Диски`

Рассмотрим их по порядку:

#### Электростеклоподъемники

In [ ]:
describe(train, 'Электростеклоподъемники')

Значения "передние и задние" и "только задние" поля `Электростеклоподъемники` трейна можно соотнести с созданнами нами в тесте полями `Электростеклоподъёмники задние::Комфорт` и `Электростеклоподъёмники передние::Комфорт` (сразу добавим эти поля в список общих полей):

In [ ]:
feature = 'Электростеклоподъёмники'
source = 'Электростеклоподъемники'
group = 'Комфорт'
for kind in ["задние", "передние"]:
    field_name = f'{feature} {kind}::{group}'
    train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and kind in x else 0)
    describe(train, field_name)
    common_cols.add(field_name)
    
    print("")

#### Усилитель руля

In [ ]:
describe(train, 'Усилитель руля')

Поле трейна `Усилитель руля` можно соотнести с созданным нами в тесте полем `Усилитель руля::Комфорт` (сразу поместим это поле в список общих колонок):

In [ ]:
feature = 'Усилитель руля'
group = 'Комфорт'
source = feature
field_name = f'{feature}::{group}'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) else 0)
describe(train, field_name)
common_cols.add(field_name)

#### Аудиосистема

Поле `Аудиосистема` трейна не стыкуется с созданным нами в тесте полями `Аудиосистема Hi-Fi::Мультимедиа`, `Аудиосистема с TV::Мультимедиа`, `Аудиосистема::Мультимедиа`:

In [ ]:
describe(train, 'Аудиосистема')

#### Фары

In [ ]:
describe(train, 'Фары')

Значение "ксеноновые" поля `Фары` трейна можно соотнести с полем `Ксеноновые/Биксеноновые фары::Обзор` созданным нами в тесте, а значение "светодиодные" - с полем `Светодиодные фары::Обзор` (сразу занесем использованные поля в список общих полей):

In [ ]:
source = 'Фары'
group = 'Обзор'

feature = 'Ксеноновые/Биксеноновые фары'
field_name = f'{feature}::{group}'
mark = 'ксеноновые'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)
print("")

feature = 'Светодиодные фары'
field_name = f'{feature}::{group}'
mark = 'светодиодные'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)


#### Климат-контроль

In [ ]:
describe(train, 'Климат-контроль')

Значение "климат-контроль многозонный" можно соотнести с полем `Климат-контроль многозонный::Комфорт`, а значение "климат-контроль однозонный" - с полем `Климат-контроль 1-зонный::Комфорт` (сразу занесем эти поля в список общих):

In [ ]:
source = 'Климат-контроль'
group = 'Комфорт'

feature = 'Климат-контроль многозонный'
field_name = f'{feature}::{group}'
mark = 'многозонный'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)
print("")

feature = 'Климат-контроль 1-зонный'
field_name = f'{feature}::{group}'
mark = 'однозонный'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)


#### Салон

In [ ]:
describe(train, 'Салон')

Значения можно соотнести следующим образом:
    - "ткань" => `Ткань (Материал салона)::Салон`
    - "кожа" => `Кожа (Материал салона)::Салон`
    - "велюр" => `Велюр (Материал салона)::Салон`
    - "комбинированный" => `Комбинированный (Материал салона)::Салон`
Выполним это соответстие, сохранив использованные поля в списке общих полей:

In [ ]:
source = 'Салон'
group = 'Салон'

feature = 'Ткань (Материал салона)'
field_name = f'{feature}::{group}'
mark = 'ткань'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)
print("")

feature = 'Кожа (Материал салона)'
field_name = f'{feature}::{group}'
mark = 'кожа'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)
print("")

feature = 'Велюр (Материал салона)'
field_name = f'{feature}::{group}'
mark = 'велюр'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)
print("")

feature = 'Комбинированный (Материал салона)'
field_name = f'{feature}::{group}'
mark = 'комбинированный'
train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
common_cols.add(field_name)
describe(train, field_name)

#### Диски

In [ ]:
describe(train, 'Диски')

Значения можно соотнести следующим образом:
    - '14"' => `Диски 14::Элементы экстерьера`
    - '15"' => `Диски 15::Элементы экстерьера`
    - '16"' => `Диски 16::Элементы экстерьера`
    - '17"' => `Диски 17::Элементы экстерьера`
    - '18"' => `Диски 18::Элементы экстерьера`
    - '19"' => `Диски 19::Элементы экстерьера`
    - '20"' => `Диски 20::Элементы экстерьера`
    - '21"' => `Диски 21::Элементы экстерьера`
    - '22"' => `Диски 22::Элементы экстерьера`
Выполним это соответстие, сохранив использованные поля в списке общих полей:

In [ ]:
source = 'Диски'
group = 'Элементы экстерьера'

for i in range(14,23):
    feature = f'Диски {i}'
    field_name = f'{feature}::{group}'
    mark = str(i)
    train[field_name] = train[source].apply(lambda x: 1 if pd.notna(x) and mark in x else 0)
    common_cols.add(field_name)
    describe(train, field_name)

### Привод

In [ ]:
field_name = 'Привод'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)

### Руль

In [ ]:
field_name = 'Руль'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)

### Состояние

In [ ]:
field_name = 'Состояние'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Здесь совершенно про разное, машина может не требовать ремонта, но при этом быть "битой" (ранее попадала в аварию) и "не битой"

При этом "битые" машины мы вряд ли будем выкупать, поэтому "битые даже не должны доходить до нашей модели

Исключим все "битые" из трейна, но колонку "Состояние" в список общих колонок добавлять не будем, так как не информативна в данном случае

In [ ]:
train = train[ train[field_name].map(lambda x: 1 if pd.notna(x) and x == "Битый" else 0) == 0 ]

In [ ]:
describe(train, field_name)

In [ ]:
train.info(verbose=True)

### Владельцы

In [ ]:
field_name = 'Владельцы'
describe(df, field_name)

In [ ]:
describe(train, field_name)

Приведем значения датасетов к общему знаменателю: "1", "2", "3":

In [ ]:
df[field_name] = df[field_name].apply(lambda x: x[0])
train[field_name] = train[field_name].apply(lambda x: None if pd.isna(x) else x[0] if int(x[0]) < 4 else "3")
describe(df, field_name)
describe(train, field_name)

Добавим рассматриваемую колонку в список колонок единого датасета:

In [ ]:
common_cols.add(field_name)

### ПТС

In [ ]:
field_name = 'ПТС'
describe(df, field_name)

Аналогов в трейне не обнаружено

### Таможня

In [ ]:
field_name = 'Таможня'
describe(df, field_name)

Аналогов в трейне не обнаружено

### Владение

In [ ]:
field_name = 'Владение'
describe(df, field_name)
df[field_name].unique()

Преобразуем в количество месяцев:

In [ ]:
def transform(x):
    if x is None:
        return None
    else:
        found_month = re.search('^(\d+) месяц(?:а|ев)?', x)
        if found_month:
            return found_month.group(1)   
        else:
            found_year_month = re.search('^(\d+) (?:года?|лет)(?: и (\d+) месяц(?:а|ев)?)?', x)
            if found_year_month is None:
                print(x)
                return None
            else:
                if found_year_month.group(2) is None:                   
                    return int(found_year_month.group(1)) * 12
                else:
                    return int(found_year_month.group(1)) * 12 + int(found_year_month.group(2))
df[field_name] = df[field_name].apply(lambda x: None if pd.isna(x) else transform(x))

Аналогов в трейне не обнаружено

## Сведение в единый датасет



Проверим состав общих полей:

In [ ]:
print("Общих полей:", len(common_cols))
print("------------")
for field_name in sorted(list(common_cols)):
    print(field_name)

Добавим в трейн и в тест поле-маркер `is_train`, добавив это поле в список общих полей:

In [ ]:
field_name = 'is_train'
test[field_name] = 0
train[field_name] = 1
common_cols.add(field_name)

Добавим в тестовый датасет поле `price`, поместим в это же поле значение целевой переменной в трейне из поля `itemPrice`, и добавим поле `price` в список общих полей:

In [ ]:
field_name = 'price'
test[field_name] = None
train[field_name] = train['itemPrice']
common_cols.add(field_name)

и объедим два датасета в один:

In [ ]:
common_cols = sorted(list(common_cols))
common_df_orig = test[common_cols].append(train[common_cols], sort=False).reset_index(drop=True)
print(common_df_orig.info(verbose=True))
df = common_df_orig

## Feature engineering

Прежде, чем мы приступим к обучению модели, добавим еще несколько features нашему датасету:

### Налог

Добавим колонку "Налог", в котором укажем размер ежегодного налога на автомобиль согласно https://glavkniga.ru/situations/s509668:

In [ ]:
describe(df, 'fuelType')

In [ ]:
field_name = 'Налог'
def calc_tax(x):
    x = int(x)
    return x*12 if x<=100 else x*25 if x>100 and x<=125 else x*35 if x>125 and x<=150 else x*45 if x>150 and x<=175 else x*55 if x>175 and x<=200 else x*65 if x>200 and x<=225 else x*75 if x>225 and x<=250 else x*150
df[field_name] = df.apply(lambda row: 
                          0 if row['fuelType'] == 'электро' else 
                          None if pd.isna(row['enginePower']) else calc_tax(row['enginePower']), 
                            axis=1
                         )
# df[]
describe(df, field_name)

### Время эксплуатации

In [ ]:
field_name = 'Время эксплуатации'
source = 'productionDate'
df[field_name] = 2021 - df[source]
describe(df, field_name)

### Средний пробег

In [ ]:
field_name = 'Средний пробег'
df[field_name] = df['mileage'].astype(float) / df['Время эксплуатации'].astype(float)

### Преобразование типов

In [ ]:
field_name = 'engineDisplacement'
df[field_name] = df[field_name].apply(lambda x: 0 if pd.isna(x) or x == 'undefined' else int(float(x) * 10))
describe(df, field_name)

In [ ]:
field_name = 'price'
df[field_name] = df[field_name].fillna(0).astype(np.int)

field_name = 'engineDisplacement'

df = df.dropna()

for field_name in [
    'enginePower', 
    'mileage', 
    'numberOfDoors', 
    'productionDate', 
    'Владельцы',
    'Налог', 
    'Время эксплуатации', 
    'Средний пробег'
]:
    df[field_name] = df[field_name].astype(np.int)

In [ ]:
common_df_orig = df

## Подготовка submission'a для kaggle

Определим функцию для подготовки submission для kaggle:

In [ ]:
sample_submission_url = "https://drive.google.com/u/0/uc?id=1XktGbf7aLmAd_eyTBL0YGYGXV8WLN5e1&export=download"
sample_submission_orig = pd.read_csv(sample_submission_url)
def make_submission(model, version, tag):
    predict_submission = model.predict(test)
    sample_submission = sample_submission_orig.copy()
    sample_submission['price'] = predict_submission
    sample_submission.price = sample_submission.price.astype(np.int32)
    sample_submission.to_csv(f'submission_v{version}_{tag}.csv', index=False)
    sample_submission.head(10)

## Оценка модели

Определим функции для оценки модели

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred1 = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

def print_regression_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'RMSE = {rmse:.2f}, MAE = {mae:.2f}, R-sq = {r2:.2f}, MAPE = {mape:.2f} ')

## Случайный лес

Попробуем использовать для обучения модели Random Forest

Применим label encoding ко всем строковым поля:

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df = common_df_orig.copy()
for field_name in ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission', 'Привод', 'Руль', 'model']:
    v = df[field_name].values.tolist()
    encoder.fit(v)
    df[field_name] = encoder.transform(df[field_name])
    describe(df, field_name)

Разъединяем датасеты:

In [ ]:
train = df.query('is_train==1').drop(['is_train'], axis = 1)
test = df.query('is_train==0').drop(['is_train', 'price'], axis = 1)

Подготовим выборки для обучения и проверки:

In [ ]:
VAL_SIZE = 0.3
RANDOM_SEED = 77
from sklearn.model_selection import train_test_split
target_field_name = 'price'
X = train.drop([target_field_name],axis=1)
y = train[target_field_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = VAL_SIZE, random_state=RANDOM_SEED)


Обучим модель:

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from collections import defaultdict
from sklearn.ensemble import RandomForestRegressor

RANDOM_SEED = 42
model = RandomForestRegressor(n_estimators=30, max_features=10,  max_depth = 20, random_state=RANDOM_SEED) #max_depth = 15, min_samples_leaf = 1, min_samples_split = 5, n_estimators = 100, verbose = 100)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print_regression_metrics(y_test, y_pred)

Подготовим submission для kaggle:

In [ ]:
make_submission(model, VERSION, 'random_forest')

## LightGBM

Попробуем использовать для обучения модели LightGBM

Применим label encoding ко всем строковым поля:

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df = common_df_orig.copy()
for field_name in ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission', 'Привод', 'Руль', 'model']:
    v = df[field_name].values.tolist()
    encoder.fit(v)
    df[field_name] = encoder.transform(df[field_name])
    describe(df, field_name)

LightGBM требует только английские буквы в названии колонок:

In [ ]:
# https://stackoverflow.com/questions/14173421/use-string-translate-in-python-to-transliterate-cyrillic
symbols = (u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ :()",
           u"abvgdeejzijklmnoprstufhzcss_y_euaABVGDEEJZIJKLMNOPRSTUFHZCSS_Y_EUA____")
tr = {ord(a):ord(b) for a, b in zip(*symbols)}
rename_map = {}
for col in df.columns.tolist():
    field_name = col.translate(tr)
    rename_map[col] = col.translate(tr)
df.rename(columns=rename_map, inplace=True)


Разъединяем датасеты:

In [ ]:
train = df.query('is_train==1').drop(['is_train'], axis = 1)
test = df.query('is_train==0').drop(['is_train', 'price'], axis = 1)

Подготовим данные для обучения и проверки:

In [ ]:
VAL_SIZE = 0.3
RANDOM_SEED = 77
from sklearn.model_selection import train_test_split
target_field_name = 'price'
X = train.drop([target_field_name],axis=1)
y = train[target_field_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = VAL_SIZE, random_state=RANDOM_SEED)


Обучим модель:

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRegressor(random_state=RANDOM_SEED)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print_regression_metrics(y_test, y_pred)

Подготовим submission для kaggle:

In [ ]:
make_submission(model, VERSION, 'ligthgbm')

## Catboost

Для обучения модели будем использовать [Catboost](https://catboost.ai/):

Сначала разъединим датасеты:

In [ ]:
df = common_df_orig.copy()
train = df.query('is_train==1').drop(['is_train'], axis = 1)
test = df.query('is_train==0').drop(['is_train', 'price'], axis = 1)

Подготовим выборки для обучения и проверки:

In [ ]:
from sklearn.model_selection import train_test_split

VAL_SIZE   = 0.30
RANDOM_SEED = 42

target_field_name = 'price'
X = train.drop([target_field_name],axis=1)
y = train[target_field_name]
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size = VAL_SIZE, random_state=RANDOM_SEED)

Обучим модель:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics 
from catboost import CatBoostRegressor

# CATBOOST
ITERATIONS = 10000
LR         = 0.1
cat_features_ids = np.where(X_train.apply(pd.Series.nunique) < 3000)[0].tolist()
model = CatBoostRegressor(iterations = ITERATIONS,
                          learning_rate = LR,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE']
                         )
model.fit(X_train, y_train,
         cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [ ]:
model.save_model('catboost_single_model_baseline.model')

Получим оценки модели:

In [ ]:
y_pred = model.predict(X_test)
print_regression_metrics(y_test, y_pred)

Проверим влияние параметров на целевую переменную:

In [ ]:
features_importances = pd.DataFrame(data = model.feature_importances_, index = X.columns, columns = ['FeatImportant'])
features_importances.sort_values(by = 'FeatImportant', ascending = False).head(20)

Подготовим submission для kaggle:

In [ ]:
make_submission(model, VERSION, 'catboost')

## Blending

Дополнительно применим блендинг:

In [ ]:
N_FOLDS    = 10
from sklearn.model_selection import KFold
from tqdm import tqdm

def cat_model(y_train, X_train, X_test, y_test):
    model = CatBoostRegressor(iterations = ITERATIONS,
                              learning_rate = LR,
                              eval_metric='MAPE',
                              random_seed = RANDOM_SEED,)
    model.fit(X_train, y_train,
              cat_features=cat_features_ids,
              eval_set=(X_test, y_test),
              verbose=False,
              use_best_model=True,
              plot=False)
    return(model)
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))
sample_submission = sample_submission_orig.copy()

submissions = pd.DataFrame(0,columns=["sub_1"], index=sample_submission.index) # куда пишем предикты по каждой модели
score_ls = []
splits = list(KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED).split(X, y))
for idx, (train_idx, test_idx) in tqdm(enumerate(splits), total=N_FOLDS,):
    # use the indexes to extract the folds in the train and validation data
    X_train, y_train, X_test, y_test = X.iloc[train_idx], y.iloc[train_idx], X.iloc[test_idx], y.iloc[test_idx]
    # model for this fold
    model = cat_model(y_train, X_train, X_test, y_test,)
    # score model on test
    test_predict = model.predict(X_test)
    test_score = mape(y_test, test_predict)
    score_ls.append(test_score)
    print(f"{idx+1} Fold Test MAPE: {mape(y_test, test_predict):0.3f}")
    # submissions
    submissions[f'sub_{idx+1}'] = model.predict(test)
    model.save_model(f'catboost_fold_{idx+1}.model')
print(f'Mean Score: {np.mean(score_ls):0.3f}')
print(f'Std Score: {np.std(score_ls):0.4f}')
print(f'Max Score: {np.max(score_ls):0.3f}')
print(f'Min Score: {np.min(score_ls):0.3f}')

Получим оценки модели:

In [ ]:
y_pred = model.predict(X_test)
print_regression_metrics(y_test, y_pred)

Подготовим submission для kaggle:

In [ ]:
make_submission(model, VERSION, 'catboost_blended')